In [2]:
!pip install geopy
!pip install openpyxl
!pip install pandas
!pip install requests

   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 667.8 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.0 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 61.0/61.0 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 3.9 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/11.6 MB 6.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.6 MB 10.5 MB/s eta 0:00:01
   -------- ----------

In [17]:
import pandas as pd

df = pd.read_excel('locations/combined_excel_updated.xlsx')
df

df.to_csv('locations/combined_excel_updated.csv', index=False)

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

# Load the Excel file
df = pd.read_excel('combined.xlsx')

# Extract address, postal code, city, and country information
addresses = df['Adresse'] + ', ' + df['Postleitzahl'] + ', ' + df['Ort'] + ', ' + df['Land']

# Initialize the Nominatim geolocator
geolocator = Nominatim(user_agent="myGeocoder")

# Create empty lists to store latitude and longitude
latitudes = []
longitudes = []

# Loop through each address and retrieve latitude and longitude
for address in addresses:
    try:
        location = geolocator.geocode(address)
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)
    except:
        latitudes.append(None)
        longitudes.append(None)

# Create new columns in the DataFrame to store latitude and longitude
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [13]:
#combine all adress columns to one
import pandas as pd
df = pd.read_excel(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\gis\locations\Amadeus_Export_3.xlsx')
print(df.columns)

df['Full Address'] = df['Adresse'].astype(str) + ', ' + df['Postleitzahl'].astype(str) + ', ' + df['Region'] + ', ' + df['Land']
df.to_csv(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\gis\locations\location_amadeus.csv', index=False)

Index(['Unnamed: 0', 'Name des Unternehmens',
       'Anzahl der Mitarbeiter \nLetztes verf. Jahr', 'Adresse',
       'Postleitzahl', 'Ort', 'Region', 'Land', 'Länderkürzel'],
      dtype='object')


In [14]:
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Replace 'YOUR_API_KEY' with your actual Geoapify API key
api_key = 'edb8cde1acdc48f39b75476ff37d05f3'

# Load the csv files
df = pd.read_csv(r'C:\Users\André\Desktop\Universität\Universität\Master\Masterarbeit\gis\locations\location_amadeus.csv')

# Define the API endpoint
base_url = 'https://api.geoapify.com/v1/geocode/search'

# Define the headers for the API request
headers = CaseInsensitiveDict()
headers['Accept'] = 'application/json'

# Initialize empty lists to store latitude and longitude
latitudes = []
longitudes = []

# Loop through each address to retrieve latitude and longitude
for index, row in df.iterrows():
    address = row['Full Address']

    if address == "nan":
        latitudes.append(None)
        longitudes.append(None)
        continue

    # Define the parameters for the API request
    params = {
        'text': address,
        'apiKey': api_key,
        'lang': 'en',  # Set the language to English
        'limit': 1,  # Set the limit to 1 to get the first result
        'type': 'amenity'  # Set the location type to amenity (you can adjust this based on your needs)
    }

    # Send the API request
    response = requests.get(base_url, params=params, headers=headers)
        # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        response_data = response.json()

        # Check if 'features' list is not empty
        if len(response_data['features']) > 0:
            # Extract latitude and longitude from the response
            latitude = response_data['features'][0]['geometry']['coordinates'][1]
            longitude = response_data['features'][0]['geometry']['coordinates'][0]

            # Append latitude and longitude to the lists
            latitudes.append(latitude)
            longitudes.append(longitude)

            # Print the address and latitude/longitude for each successful request
            print(f"Address: {address}")
            print(f"Latitude: {latitude}")
            print(f"Longitude: {longitude}")
            print()
        else:
            print(f"No results found for address: {address}")
            latitudes.append(None)
            longitudes.append(None)
    else:
        print(f"Error geocoding address: {address}")
        latitudes.append(None)
        longitudes.append(None)

# Create new columns in the DataFrame to store latitude and longitude
df['Latitude'] = latitudes
df['Longitude'] = longitudes


Address: WEENA 505, 3013 AL, Rotterdam, Niederlande
Latitude: 51.9237047
Longitude: 4.471875

Address: 75 QUAI D ORSAY, 75007, Paris, Frankreich
Latitude: 48.8622331
Longitude: 2.3047133

Address: BOTLEKWEG 121 HAVENNUMMER, 3197 KA, Rotterdam, Niederlande
Latitude: 51.870317
Longitude: 4.300543

Address: MARKGRAFENSTR. 23, 10117, Berlin, Deutschland
Latitude: 52.5083233
Longitude: 13.3943543

Address: DRAMMENSVEIEN 264, 0283, Oslo, Norwegen
Latitude: 59.913821
Longitude: 10.650463

Address: SUHRENKAMP 71-77, 22335, Hamburg, Freie und Hansestadt, Deutschland
Latitude: 53.6212995
Longitude: 10.0152091

Address: RANSBEEKSTRAAT 310, 1120, Brussels-Capital Region, Belgien
Latitude: 50.901131
Longitude: 4.400383

Address: DUNAJSKA CESTA 50, 1000, Osrednjeslovenska, Slovenien
Latitude: 46.0647094
Longitude: 14.5092881

Address: CAFFAMACHERREIHE 1, 20355, Hamburg, Freie und Hansestadt, Deutschland
Latitude: 53.5543815
Longitude: 9.9844534

Address: 11 13 TOUR PACIFIC, 11 COURS VALMY, 92800, Ha

In [15]:
df.to_csv('locations/location_amadeus.csv', index=False)

In [20]:
!pip install chardet
import csv
import chardet
# List to store the cleaned data
clean_data = []

# Open the CSV file with 'rb' mode to read in binary mode
with open(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\webcrawler\data\france_member.csv', 'rb') as file:
    # Use chardet to detect the encoding of the file
    result = chardet.detect(file.read())
    encoding = result['encoding']

# Open the CSV file with the detected encoding
with open(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\webcrawler\data\france_member.csv', 'r', encoding=encoding) as file:
    reader = csv.reader(file)

    # Skip the header row
    next(reader)

    # Iterate through each row in the CSV file
    for row in reader:
        # Split the address by newline character(s)
        address_parts = row[2].replace('\r', '').split('\n')

        # Clean and join the address parts
        cleaned_address = ' '.join(address_parts)

        # Create a new row with the cleaned address
        new_row = [row[0], row[1], cleaned_address]

        # Add the new row to the clean_data list
        clean_data.append(new_row)

# Open a new CSV file to write the cleaned data
with open(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\webcrawler\data\france_member_cleaned.csv', 'w', newline='', encoding=encoding) as new_file:
    writer = csv.writer(new_file)

    # Write the header row
    writer.writerow(['Name', 'Representative', 'Address'])

    # Write the clean_data rows
    writer.writerows(clean_data)

   ---------------------------------------- 0.0/199.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/199.4 kB ? eta -:--:--
   ------ -------------------------------- 30.7/199.4 kB 445.2 kB/s eta 0:00:01
   ---------------------------------- ----- 174.1/199.4 kB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 199.4/199.4 kB 1.7 MB/s eta 0:00:00


In [25]:
import pandas as pd
df1 = pd.read_excel('C:\\Users\\osi0pr\\Documents\\Github\\Masterarbeit\\gis\\data\\data-krTLV.xlsx')
df2 = pd.read_excel('C:\\Users\\osi0pr\\Documents\\Github\\Masterarbeit\\gis\\data\\data-MDu07 .xlsx')


In [26]:
df1['Latitude'] = df1['LAT']
df1['Longitude'] = df1['LON']
df2['Latitude'] = df2['Lat']
df2['Longitude'] = df2['Lon']

In [28]:
df2.drop(['Lat', 'Lon', 'Title', 'Column 1'], axis=1, inplace=True)
df2

,Partner,Partner ohne Markdown,Website,Straße,PLZ,Ort,Land,Partner aus…,Beteiligt in…,Funktion als…,Webadresse,Latitude,Longitude
0,[ABB AG](https://www.abb.de/),ABB AG,https://www.abb.de/,Kallstadter Str. 1,68309,Mannheim,Baden-Württemberg,Wirtschaft,FertiRob,Partner,www.abb.de,49.489291,8.4673098
1,[Advanced Training Technologies GmbH](https://...,Advanced Training Technologies GmbH,https://sk-att.com/,Am Förderturm 5 c,44575,Castrop-Rauxel,Nordrhein-Westfalen,Wirtschaft,HyPLANT100,Partner,sk-att.com,51.564619,7.3106175
2,[Alantum Europe GmbH](https://www.alantum.com/),Alantum Europe GmbH,https://www.alantum.com/,Balanstr. 73 / Geb. 21A,81541,München,Bayern,Wirtschaft,AEL4GW,Partner,www.alantum.com,48.259248,11.4354419
3,[Albert-Ludwigs-Universität Freiburg](https://...,Albert-Ludwigs-Universität Freiburg,https://www.uni-freiburg.de/,Georges-Köhler-Allee 103,79110,Freiburg im Breisgau,Baden-Württemberg,Wissenschaft,Fluorfreie MEA,Partner,www.uni-freiburg.de,47.965146,7.9573306
4,[AUDI Aktiengesellschaft](https://www.audi.de/),AUDI Aktiengesellschaft,https://www.audi.de/,Auto-Union-Str. 1,85057,Ingolstadt,Bayern,Wirtschaft,PEP.IN,Partner,www.audi.de,48.763016,11.4250395
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,[Zentrum für Brennstoffzellen-Technik GmbH](ht...,Zentrum für Brennstoffzellen-Technik GmbH,https://www.zbt.de/,Carl-Benz-Str. 201,47057,Duisburg,Nordrhein-Westfalen,Wissenschaft,"AlFaKat, Degrad-EL3, PEP.IN",Koordinator und Partner,www.zbt.de,51.333120,6.5623343
113,[Öko-Institut. Institut für angewandte Ökologi...,Öko-Institut. Institut für angewandte Ökologie...,https://www.oeko.de/,Rheinstr. 95,64295,Darmstadt,Hessen,Wissenschaft,ReNaRe,Partner,www.oeko.de,49.872775,8.651177
114,[Semodia GmbH](https://www.semodia.com),Semodia GmbH,https://www.semodia.com,Meißner Str. 37,1445,Radebeul,Sachsen,Wirtschaft,eModule,Partner,www.semodia.com,51.097174,13.6923622
115,[Semodia GmbH](https://www.semodia.com),Semodia GmbH,https://www.semodia.com,Meißner Str. 37,1445,Radebeul,Sachsen,Wirtschaft,eModule,Partner,www.semodia.com,52.194911,11.5985197


In [29]:
df1.drop(['LAT', 'LON', 'Title', 'Mehrere Partner'], axis=1, inplace=True)

In [30]:
frames = [df1, df2]
merge_df = pd.concat(frames)

merge_df
merge_df.to_csv('deutschland_projecte.csv', index=False)

In [86]:
# Filter the DataFrame to show rows where the 'Partner' column contains NaN values
nan_rows = df1[df1['Partner'].isnull()]

# Print the filtered DataFrame
print(nan_rows)

Empty DataFrame
Columns: [Lat, Lon, Title, Partner, Partner ohne Markdown, Website, Straße, PLZ, Ort, Land, Partner aus…, Beteiligt in…, Funktion als…, Webadresse, Column 1, lat, lon]
Index: []


In [78]:
df1.loc[12,'Lat']

'50.1427962,9.0464884,,De Nora Deutschland GmbH,De Nora Deutschland GmbH,https://www.denora.com,Industriestr. 17,63517,Rodenbach,Hessen,Wirtschaft,"INSTALL-AWE, PrometH2eus",Partner,www.denora.com,'

In [1]:
import pandas as pd
df = pd.read_excel("locations/combined_excel.xlsx")
df.to_csv("locations/combined_data.csv", index=False)

In [6]:

df = pd.read_csv('locations/combined_excel_updated.csv')
df



,Lat,Lon,Name,Full Address,Address,ZIP,City,Country,Source
0,50.299470,8.271285,"""GERMAN ASSOCIATION FOR GAS TECHNOLOGY"" E.V.","WEIßERDSTR., 65520, Limburg-Weilburg, Deutschland",WEIßERDSTR.,65520,BAD CAMBERG,Germany,Amadeus
1,47.487849,12.078639,"""INTERCHIM AUSTRIA"" INTERNATIONALER CHEMIEHAND...","BRIXENTALERSTRAßE 67, 6300, Tirol, Österreich",BRIXENTALERSTRAßE 67,6300,WÖRGL,Austria,Amadeus
2,36.339571,28.204710,(SOLMAR),"A VIPE 0 IRAKLIOU, NEA ALIKARNASSOS, HERAKLION...",NaN,NaN,NaN,Greece,Members locations | Hydrogen Europe
3,NaN,NaN,?ukasiewicz – Institute of Electrical Engineer...,"Mieczysława Pożaryskiego 28, 04-703 Warszawa, ...",Mieczys?awa Po?aryskiego 28,04-703 Warsaw,NaN,NaN,Hydrogen investment map - Portal wodorowy (h2p...
4,38.728847,-9.143946,"1s1 Energy Portugal, Unipessoal, Lda","Av. Duque de Loulé 5 6ºC, Lisbon, 1050-085, Po...",NaN,NaN,NaN,Portugal,Members locations | Hydrogen Europe
...,...,...,...,...,...,...,...,...,...
4038,57.757060,29.364888,ЧАСТНОЕ ЛЕЧЕБНО-ПРОФИЛАКТИЧЕСКОЕ УЧРЕЖДЕНИЕ СА...,"УЛ. ЦЕНТРАЛЬНАЯ Д. 1, 182651, Псковская област...",УЛ. ЦЕНТРАЛЬНАЯ Д. 1,182651,Д. ХИЛОВО,Russia,Amadeus
4039,42.691527,23.353074,ШЕЛ БЪЛГАРИЯ,"СЕРДИКА ОФИСИ, БУЛ. СИТНЯКОВО 48, ЕТ. 8, 1505,...","СЕРДИКА ОФИСИ, БУЛ. СИТНЯКОВО 48, ЕТ. 8",1505,СОФИЯ,Bulgaria,Amadeus
4040,55.728682,37.645809,ЭВОНИК ХИМИЯ,"УЛ. КОЖЕВНИЧЕСКАЯ Д. 14, СТР. 5, 115114, г.Мос...","УЛ. КОЖЕВНИЧЕСКАЯ Д. 14, СТР. 5",115114,МОСКВА,Russia,Amadeus
4041,55.797550,38.186378,ЭСТАФЕТА,"УЛ.КИРОВА Д.29, 142450, Московская область, Ru...",УЛ.КИРОВА Д.29,142450,СТАРАЯ КУПАВНА,Russia,Amadeus


In [8]:
#print df row where column "Full Address" is empty

import pandas as pd

# Filter the DataFrame to show rows where the 'Source' column is equal to "Wasserstoff-Leitprojekte: Startseite"
if 'Wasserstoff-Leitprojekte: Startseite' in df['Source'].tolist():
    # Fill "Full Address" with a combination of "Address", "ZIP", "City", and "Country" for the selected rows
    df.loc[df['Source'] == 'Wasserstoff-Leitprojekte: Startseite', 'Full Address'] = df['Address'].fillna('')\
                                                         .astype(str) + ', ' + df['ZIP'].fillna('')\
                                                         .astype(str) + ', ' + df['City'] + ', ' + df['Country']

# Filter the DataFrame to show rows where the 'Source' column is equal to "Wasserstoff-Leitprojekte: Startseite"
filtered_df = df[df['Source'] == 'Wasserstoff-Leitprojekte: Startseite']
filtered_df




In [23]:
import pandas as pd
import requests
from requests.structures import CaseInsensitiveDict

# Replace 'YOUR_API_KEY' with your actual Geoapify API key
api_key = 'edb8cde1acdc48f39b75476ff37d05f3'

# Load the csv files
df = pd.read_csv(r'C:\Users\osi0pr\Documents\Github\Masterarbeit\gis\locations\combined_excel_updated.csv')

# Define the API endpoint
base_url = 'https://api.geoapify.com/v1/geocode/search'

# Define the headers for the API request
headers = CaseInsensitiveDict()
headers['Accept'] = 'application/json'

# Initialize empty lists to store latitude and longitude
latitudes = []
longitudes = []

# Loop through each address to retrieve latitude and longitude
for index, row in df.iterrows():
    address = row['Full Address']

    if isinstance(address, float):
        print(f"Address: {address}")
        print("NaN Value - nothing changed")
        continue

    # Define the parameters for the API request
    params = {
        'text': address,
        'apiKey': api_key,
        'lang': 'en',  # Set the language to English
        'limit': 1,  # Set the limit to 1 to get the first result
        'type': 'amenity'  # Set the location type to amenity (you can adjust this based on your needs)
    }

    # Send the API request
    response = requests.get(base_url, params=params, headers=headers)
        # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        response_data = response.json()

        # Check if 'features' list is not empty
        if len(response_data['features']) > 0:
            # Extract latitude and longitude from the response
            latitude = response_data['features'][0]['geometry']['coordinates'][1]
            longitude = response_data['features'][0]['geometry']['coordinates'][0]

            # Append latitude and longitude to the lists
            latitudes.append(latitude)
            longitudes.append(longitude)

            # Print the address and latitude/longitude for each successful request
            print(f"Address: {address}")
            print(f"Latitude: {latitude}")
            print(f"Longitude: {longitude}")
            print()
        else:
            print(f"No results found for address: {address}")
            continue
    else:
        print(f"Error geocoding address: {address}")
        latitudes.append(None)
        longitudes.append(None)

# Create new columns in the DataFrame to store latitude and longitude
df['Latitude'] = latitudes
df['Longitude'] = longitudes


Address: WEIßERDSTR., 65520, Limburg-Weilburg, Deutschland
Latitude: 50.29946965
Longitude: 8.27128530323418

Address: BRIXENTALERSTRAßE 67, 6300, Tirol, Österreich
Latitude: 47.48784935
Longitude: 12.078638748864481

Address: A VIPE 0 IRAKLIOU, NEA ALIKARNASSOS, HERAKLION - CRETE, 716 01, Greece
Latitude: 36.3395715
Longitude: 28.204709603067563

Address: Mieczysława Pożaryskiego 28, 04-703 Warszawa, Poland
Latitude: 52.2071571
Longitude: 21.173925581191433

Address: Av. Duque de Loulé 5 6ºC, Lisbon, 1050-085, Portugal
Latitude: 38.7288472
Longitude: -9.1439462

No results found for address: Unit 1 Sycamore Ct, Warrington Rd, Runcorn WA7 1RS, United Kingdom
Address: BENZSTR. 3, 48619, Borken, Deutschland
Latitude: 52.11571435
Longitude: 7.083733369217654

Address: Piekarska 86, 43-300 Bielsko-Bia?a
Latitude: 49.832097
Longitude: 19.065788

Address: 199 RUE DE LA BOUGRIERE- 44980 SAINTE LUCE SUR LOIRE -
Latitude: 47.2647203
Longitude: -1.4766609

Address: 4b rue François de Curel 57000